# Create a Project
Note:  This requires that you have enabled products within SageMaker Studio

![](img/enable-service-catalog-portfolio-for-studio.png)

In [ ]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
sc = boto3.Session().client(service_name='servicecatalog', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [ ]:
import time
timestamp = int(time.time())

In [ ]:
search_response = sc.search_products(
   Filters={
       'FullTextSearch': 
       [
           'MLOps template for model building, training, and deployment'
       ]
   }
)

sagemaker_pipeline_product_id = search_response['ProductViewSummaries'][0]['ProductId']
print(sagemaker_pipeline_product_id)

# pprint(search_response)

In [ ]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response['ProvisioningArtifacts'][0]['Id']

pprint(describe_response)

In [ ]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

In [ ]:
sagemaker_project_name = 'dsoaws-{}'.format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription='dsoaws-{}'.format(timestamp),
    ServiceCatalogProvisioningDetails={
        'ProductId': sagemaker_pipeline_product_id,
        'ProvisioningArtifactId': sagemaker_pipeline_product_provisioning_artifact_id
     }
)

sagemaker_project_id = create_response['ProjectId']
sagemaker_project_arn = create_response['ProjectArn']

pprint(create_response)

In [ ]:
sagemaker_project_name_and_id = '{}-{}'.format(sagemaker_project_name, sagemaker_project_id)

print(sagemaker_project_name_and_id)

## Attach Feature Store Permssions To Service Catalog User Role 

### (used for Code Build Pipeline Executions)

In [ ]:
sc_role_name='AmazonSageMakerServiceCatalogProductsUseRole'

In [ ]:
account_id = sts.get_caller_identity()['Account']
print(account_id)

In [ ]:
sc_role_arn = 'arn:aws:iam::{}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'.format(account_id)
print(sc_role_arn)

In [ ]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

print(response)

In [ ]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFeatureStoreAccess'
)

print(response)

# Describe Project

In [ ]:
response = sm.describe_project(ProjectName=sagemaker_project_name)
print(response)

In [ ]:
project_status = response['ProjectStatus']
print(project_status)

# Create Project
# _Wait for Project to be Created_

In [ ]:
import time

try:
    response = sm.describe_project(ProjectName=sagemaker_project_name)
    project_status = response['ProjectStatus']
    print('Creating Project...')

    while project_status in ['Pending', 'CreateInProgress']:
        print('Please wait...')
        time.sleep(30)
        response = sm.describe_project(ProjectName=sagemaker_project_name)
        project_status = response['ProjectStatus']
        print('Project status: {}'.format(project_status))

    if project_status == 'CreateCompleted':   
        print('Project {}'.format(project_status))

    else:
        print('Project status: {}'.format(project_status))
        raise Exception('Project not created.')
        
except Exception as e:
    print(e)

# _Wait for Project to be Created ^^ Above ^^_

In [ ]:
root_path = sm_studio_root_path if os.path.isdir(sm_studio_root_path) else sm_notebooks_root_path

print(root_path)

# Clone Repos

In [ ]:
print(region)

In [ ]:
code_commit_repo1 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modelbuild'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo1)

In [ ]:
local_repo1 = '{}{}sagemaker-{}-modelbuild'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(local_repo1)

In [ ]:
code_commit_repo2 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modeldeploy'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo2)

In [ ]:
local_repo2 = '{}{}sagemaker-{}-modeldeploy'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(local_repo2)

In [ ]:
!git config --global credential.helper '!aws codecommit credential-helper $@'
!git config --global credential.UseHttpPath true

In [ ]:
!git clone $code_commit_repo1 $local_repo1

In [ ]:
!git clone $code_commit_repo2 $local_repo2

# Remove Stock `Abalone` Example Code

In [ ]:
#!rm -rf $local_repo1/pipelines/abalone

# Copy Workshop Code Into Local Project Folders

In [ ]:
workshop_project_build_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modelbuild'.format(root_path)
print(workshop_project_build_code)

In [ ]:
workshop_project_deploy_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modeldeploy'.format(root_path)
print(workshop_project_deploy_code)

In [ ]:
!cp -R $workshop_project_build_code/* $local_repo1/

In [ ]:
!cp -R $workshop_project_deploy_code/* $local_repo2/

# Commit New Code 

In [ ]:
print(local_repo1)

In [ ]:
!cd $local_repo1; git status; git add --all .; git commit -m "new code"; git push

In [ ]:
!cd $local_repo2; git status; git add --all .; git commit -m "Data Science on AWS"; git push

# Completed.

In [ ]:
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

In [ ]:
%store